# Load Data
Bilder einlesen und Dateinamen anpassen

In [1]:
# OPTIONAL: Load the "autoreload" extension so that code can change
%load_ext autoreload

# OPTIONAL: always reload modules so that as you change code in src, it gets loaded
%autoreload 2

from src.data import make_dataset

In [2]:
import pandas as pd # Deals with data

In [3]:
manual_readings = pd.read_excel('../data/raw/moroweg_strom_gas.xlsx')
manual_readings.head()

,Date,Strom kWh,Diff,Gas m^3,Diff.1,Kommentar,Unnamed: 6,Month,Strom Max,Strom Diff Previous,...,Kommentar.1,Unnamed: 21,Unnamed: 22,Strom Single kwh (Referenz),1700,%-Veränderung,Gas 50m^2 kwh (Referenz),5000,%-Veränderung.1,Unnamed: 29
0,2017-02-28,23165.80,NaN,10686.757,NaN,künstlicher Vortrag,NaN,2017-01-01,NaN,NaN,...,Ab März,NaN,NaN,2017,777.431,45.731222,2017.0,460.996,9.219926,2017 ab April
1,2017-03-04,23165.80,0.00,10686.757,0.000,Anfangsstand,NaN,2017-02-01,23165.80,NaN,...,NaN,NaN,NaN,2018,1160.14,149.227436,2018.0,791.258,171.640941,NaN
2,2017-03-18,23166.50,0.70,NaN,-10686.757,NaN,NaN,2017-03-01,23200.45,34.65,...,NaN,NaN,NaN,2019,1361.79,117.381277,2019.0,782.908,98.944661,NaN
3,2017-03-31,23200.45,33.95,10712.000,10712.000,Strom und Gas geschätzt,NaN,2017-04-01,23275.35,74.9,...,NaN,NaN,NaN,2020,475.757,34.936234,2020.0,392.426,50.124174,NaN
4,2017-04-15,23234.40,33.95,NaN,-10712.000,NaN,NaN,2017-05-01,23361.00,85.65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
## Code Snippet for exifread for a sample image
import exifread
import os

path_name = os.path.join(os.pardir, 'data', 'raw', '2017', '2017-03-03 15.06.47.jpg')

# Open image file for reading (binary mode)
f = open(path_name, 'rb')

# Return Exif tags
tags = exifread.process_file(f)

# Show Tags
for tag in tags.keys():
    if tag not in ('JPEGThumbnail', 'TIFFThumbnail', 'Filename', 'EXIF MakerNote'):
        print("Key: %s, value %s" % (tag, tags[tag]))

Key: Image Make, value OnePlus
Key: Image Model, value ONEPLUS A3003
Key: Image XResolution, value 72
Key: Image YResolution, value 72
Key: Image ResolutionUnit, value Pixels/Inch
Key: Image Software, value OnePlus3-user 7.0 NRD90M 18 dev-keys
Key: Image DateTime, value 2017:03:03 15:06:47
Key: Image YCbCrPositioning, value Centered
Key: Image ExifOffset, value 230
Key: GPS GPSAltitudeRef, value 2
Key: Image GPSInfo, value 752
Key: Thumbnail Compression, value JPEG (old-style)
Key: Thumbnail XResolution, value 72
Key: Thumbnail YResolution, value 72
Key: Thumbnail ResolutionUnit, value Pixels/Inch
Key: Thumbnail JPEGInterchangeFormat, value 872
Key: Thumbnail JPEGInterchangeFormatLength, value 8523
Key: EXIF ExposureTime, value 1/17
Key: EXIF FNumber, value 2
Key: EXIF ExposureProgram, value Unidentified
Key: EXIF ISOSpeedRatings, value 1250
Key: EXIF ExifVersion, value 0220
Key: EXIF DateTimeOriginal, value 2017:03:03 15:06:47
Key: EXIF DateTimeDigitized, value 2017:03:03 15:06:47
Key